In [39]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')


Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2019.9.11  |       hecc5488_0         144 KB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    certifi-2019.9.11          |           py36_0         147 KB  conda-forge
    openssl-1.1.1c             |       h516909a_0         2.1 MB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

    ca-

# Finding Table Using BeautifulSoup package 

In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from tabulate import tabulate

res = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(res.content,'lxml')
table = soup.find_all('table')[0] 


# extracting information from the table

In [3]:
table_rows = table.find_all('tr')
data = []
for row in table_rows:
    data.append([t.text.strip() for t in row.find_all('td')])

df = pd.DataFrame(data, columns=['PostalCode', 'Borough', 'Neighbourhood'])
df = df[~df['PostalCode'].isnull()]  # to filter out bad rows
df.drop(df[df['Borough']=="Not assigned"].index,axis=0, inplace=True) # assign Neighbourhood=Borough where Neighbourhood is 'Not assigned'
df.head()

,PostalCode,Borough,Neighbourhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights


# group multiple Neighbourhood under one Postcode

In [4]:
df1 = df.reset_index()
df2= df1.groupby('PostalCode').agg(lambda x: ','.join(x))
df2.loc[df2['Neighbourhood']=="Not assigned",'Neighbourhood']=df2.loc[df2['Neighbourhood']=="Not assigned",'Borough']
df3 = df2.reset_index()
df3['Borough']= df3['Borough'].str.replace('nan|[{}\s]','').str.split(',').apply(set).str.join(',').str.strip(',').str.replace(",{2,}",",")
df3.shape

(103, 3)

In [24]:
df3.head()

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


# PART 2

In [5]:
geo_df=pd.read_csv('http://cocl.us/Geospatial_data')#geolocator takes long time so using csv files

In [23]:
geo_df.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [18]:
geo_df.rename(columns={'Postal Code':'PostalCode'},inplace=True) #renaming column names to remove space between 'Postal Codes'
geo_merged = pd.merge(geo_df, df3, on='PostalCode')

In [25]:
geo_merged.head()

,PostalCode,Latitude,Longitude,Borough,Neighbourhood
0,M1B,43.806686,-79.194353,Scarborough,"Rouge,Malvern"
1,M1C,43.784535,-79.160497,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,43.763573,-79.188711,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,43.770992,-79.216917,Scarborough,Woburn
4,M1H,43.773136,-79.239476,Scarborough,Cedarbrae


In [31]:
geo_data=geo_merged[['PostalCode','Borough','Neighbourhood','Latitude','Longitude']] #rearrranging lattitudes and longitude columns
geo_data.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [33]:
# work with only boroughs that contain the word Toronto 
toronto_data=geo_data[geo_data['Borough'].str.contains("Toronto")]
toronto_data.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
37,M4E,EastToronto,The Beaches,43.676357,-79.293031
41,M4K,EastToronto,"The Danforth West,Riverdale",43.679557,-79.352188
42,M4L,EastToronto,"The Beaches West,India Bazaar",43.668999,-79.315572
43,M4M,EastToronto,Studio District,43.659526,-79.340923
44,M4N,CentralToronto,Lawrence Park,43.728020,-79.388790


# PART 3

In [47]:
CLIENT_ID = 'SC0F4RRIQKEO5LT3TFCDFLYOE1JCCKB3NWDGTWQPDCPMSDTF' # your Foursquare ID
CLIENT_SECRET = 'R2BNL2LV0EVF5G03GIFASMKS0ENZANFITVKVDSQOEINAIS2C' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: SC0F4RRIQKEO5LT3TFCDFLYOE1JCCKB3NWDGTWQPDCPMSDTF
CLIENT_SECRET:R2BNL2LV0EVF5G03GIFASMKS0ENZANFITVKVDSQOEINAIS2C


In [48]:
address = 'Bremner Blvd, Toronto'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

43.6404937 -79.3895815


In [49]:
search_query = 'Italian'
radius = 500
print(search_query + ' .... OK!')


Italian .... OK!


In [50]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=SC0F4RRIQKEO5LT3TFCDFLYOE1JCCKB3NWDGTWQPDCPMSDTF&client_secret=R2BNL2LV0EVF5G03GIFASMKS0ENZANFITVKVDSQOEINAIS2C&ll=43.6404937,-79.3895815&v=20180604&query=Italian&radius=500&limit=30'

In [51]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5db81bcc0f5968001bf717fd'},
 'response': {'venues': []}}

In [53]:
#assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

""
